# 面向对象高级编程

数据封装、继承和多态只是面向对象程序设计中最基础的3个概念。在Python中，面向对象还有很多高级特性，允许我们写出非常强大的功能。

## 使用__slots__

正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性。先定义class：

In [2]:
class Student(object):
    pass

In [3]:
s = Student()
s.name = 'Michael'
print(s.name)

Michael


In [4]:
def set_age(self, age): #定义一个函数作为实例方法
    self.age = age


from types import MethodType
s.set_age = MethodType(set_age, s)
s.set_age(20)
s.age

20

In [5]:
def set_score(self, score):
    self.score = score

Student.set_score = set_score

s.set_score(100)
s.score

100

通常情况下，上面的set_score方法可以直接定义在class中，但动态绑定允许我们在程序运行的过程中动态给class加上功能，这在静态语言中很难实现。

### 使用__slots__

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性。

为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的__slots__变量，来限制该class实例能添加的属性：

In [6]:
class Student(object):
    __slots__ = ('name', 'age')

In [7]:
s = Student()
s.name = 'Michael'
s.age = 23
s.score = 100


AttributeError: 'Student' object has no attribute 'score'

使用__slots__要注意，__slots__定义的属性仅对当前类实例起作用，对继承的子类是不起作用的：

In [9]:
class GraduateStudent(Student):
    pass

In [10]:
g = GraduateStudent()
g.score = 999


除非在子类中也定义__slots__，这样，子类实例允许定义的属性就是自身的__slots__加上父类的__slots__。

## 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改：

In [12]:
class Student:
    def get_score(self):
        return self._score
    
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [13]:
s = Student()
s.set_score(100)
s.get_score()

100

In [14]:
s.set_score(999)

ValueError: score must between 0 ~ 100!

但是，上面的调用方法又略显复杂，没有直接用属性这么直接简单。

有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢？对于追求完美的Python程序员来说，这是必须要做到的！

还记得装饰器（decorator）可以给函数动态加上功能吗？对于类的方法，装饰器一样起作用。Python内置的@property装饰器就是负责把一个方法变成属性调用的：

In [17]:
class Student:

    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [18]:
s = Student()
s.score = 100
s.score

100

In [19]:
s.score = 999

ValueError: score must between 0 ~ 100!

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

In [21]:
class Student:
    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value 

    @property
    def age(self):
        return 2015 - self._birth

要特别注意：属性的方法名不要和实例变量重名。

### 小结

@property广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，这样，程序运行时就减少了出错的可能性。

### 练习

请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution：

In [23]:
class Screen(object):
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        self._height = value

    @property
    def resolution(self):
        return 786432

In [24]:
# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


## 多重继承

In [25]:
class Animal(object):
    pass

# 大类:
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物:
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

In [26]:
class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')

In [27]:
class Dog(Mammal, Runnable):
    pass

In [28]:
class Bat(Mammal, Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的所有功能。

在设计类的继承关系时，通常，主线都是单一继承下来的，例如，Ostrich继承自Bird。但是，如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让Ostrich除了继承自Bird外，再同时继承Runnable。这种设计通常称之为MixIn。

为了更好地看出继承关系，我们把Runnable和Flyable改为RunnableMixIn和FlyableMixIn。类似的，你还可以定义出肉食动物CarnivorousMixIn和植食动物HerbivoresMixIn，让某个动物同时拥有好几个MixIn：

MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

Python自带的很多库也使用了MixIn。举个例子，Python自带了TCPServer和UDPServer这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由ForkingMixIn和ThreadingMixIn提供。通过组合，我们就可以创造出合适的服务来。

### 小结

由于Python允许使用多重继承，因此，MixIn就是一种常见的设计。

只允许单一继承的语言（如Java）不能使用MixIn的设计。

## 定制类

看到类似__slots__这种形如__xxx__的变量或者函数名就要注意，这些在Python中是有特殊用途的。

__slots__我们已经知道怎么用了，__len__()方法我们也知道是为了能让class作用于len()函数。

除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。

### _ _str_ _

In [1]:
class Student:
    def __init__(self, name):
        self.name = name

print(Student('Michael'))

In [2]:
class Student:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'Student Object (name: %s)' % self.name

print(Student('Michael'))

Student Object (name: Michael)


但是细心的朋友会发现直接敲变量不用print，打印出来的实例还是不好看：
这是因为直接显示变量调用的不是__str__()，而是__repr__()，两者的区别是__str__()返回用户看到的字符串，而__repr__()返回程序开发者看到的字符串，也就是说，__repr__()是为调试服务的。

解决办法是再定义一个__repr__()。但是通常__str__()和__repr__()代码都是一样的，所以，有个偷懒的写法：


In [4]:
class Student:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return 'Student Object (name: %s)' % self.name

    __repr__ = __str__

Student('Michael')

Student Object (name: Michael)

### _ _iter_ _

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的__next__()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [9]:
class Fib:
    def __init__(self):
        self.a, self.b = 1, 1 # 初始化两个计数器a，b
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 100000: # 退出循环的条件
            raise StopIteration()
        return self.a   # 返回下一个值


In [10]:
for n in Fib():
    print(n, end=',')

1,2,3,5,8,13,21,34,55,89,144,233,377,610,987,1597,2584,4181,6765,10946,17711,28657,46368,75025,

### _ _getitem_ _

Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，取第5个元素

要表现得像list那样按照下标取出元素，需要实现__getitem__()方法：

In [3]:
Fib()[5]

TypeError: 'Fib' object is not subscriptable

In [4]:
class Fib:
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b

        return a

In [7]:
f = Fib()
f[20]

10946

In [12]:
class Fib:
    def __getitem__(self, n):
        if isinstance(n, int):
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice):
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
                
            return L

In [13]:
f = Fib()
f[:10]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

也没有对负数作处理，所以，要正确实现一个__getitem__()还是有很多工作要做的。

此外，如果把对象看成dict，__getitem__()的参数也可能是一个可以作key的object，例如str。

与之对应的是__setitem__()方法，把对象视作list或dict来对集合赋值。最后，还有一个__delitem__()方法，用于删除某个元素。

总之，通过上面的方法，我们自己定义的类表现得和Python自带的list、tuple、dict没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

### _ _getattr_ _

In [ ]:
class Student:
    def __init__(self):
        self.name = 'Michael'

要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个__getattr__()方法，动态返回一个属性。

In [24]:
class Student:
    def __init__(self):
        self.name = 'Michael'
    
    def __getattr__(self, attr):
        if attr == 'score':
            return 99
        if attr == 'age':
            return lambda: 25
        raise AttributeError("'Student' object has no attribute '%s'" % attr)

In [25]:
s = Student()
print(s.name)
print(s.score)
print(s.age())
print(s.abc)

Michael
99
25


AttributeError: 'Student' object has no attribute 'abc'

注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：

这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

这种完全动态调用的特性有什么实际作用呢？作用就是，可以针对完全动态的情况作调用。

In [7]:
class Chain:
    def __init__(self, path = ''):
        self._path = path
    
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    
    def __str__(self):
        return self._path 
    
    __repr__ = __str__

In [8]:
Chain().status.user.timeline.list

/status/user/timeline/list

In [10]:
Chain().users.repos

/users/repos

### _ _call_ _

一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个__call__()方法，就可以直接对实例进行调用。

In [12]:
class Student:
    def __init__(self, name):
        self.name = name
    def __call__(self):
        print('My name is %s.' % self.name)

In [13]:
s = Student('Michael')
s()

My name is Michael.


__call__()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有__call__()的类实例

通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。

In [15]:
callable(Student('Michael'))

True

In [16]:
callable(max)

True

### 小结

Python的class允许定义许多定制方法，可以让我们非常方便地生成特定的类。

- _ _slots_ _()
- _ _len_ _()
- _ _next_ _()
- _ _str_ _()
- _ _repr_ _
- _ _iter_ _()
- _ _getitem_ _()
- _ _getattr_ _()
- _ _call_ _()

## 使用枚举类

In [30]:
from enum import Enum

Month = Enum('Moth',('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [31]:
for name, member in Month.__members__.items():
    print(name, '=>',member, ',', member.value)


Jan => Moth.Jan , 1
Feb => Moth.Feb , 2
Mar => Moth.Mar , 3
Apr => Moth.Apr , 4
May => Moth.May , 5
Jun => Moth.Jun , 6
Jul => Moth.Jul , 7
Aug => Moth.Aug , 8
Sep => Moth.Sep , 9
Oct => Moth.Oct , 10
Nov => Moth.Nov , 11
Dec => Moth.Dec , 12


value属性则是自动赋给成员的int常量，默认从1开始计数。

如果需要更精确地控制枚举类型，可以从Enum派生出自定义类：

In [32]:
from enum import Enum, unique

@unique
class WeekDay(Enum):
    Sun = 0 # Sun的value被设定为0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

@unique装饰器可以帮助我们检查保证没有重复值。

### 练习

把Student的gender属性改造为枚举类型，可以避免使用字符串：

In [33]:
from enum import Enum, unique

@unique
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

In [34]:
# 测试:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


### 小结

Enum可以把一组相关常量定义在一个class中，且class不可变，而且成员可以直接比较。

## 使用元类

TODO